In [ ]:
# Snowflake-META Onboarding Example
# This notebook demonstrates how to onboard control table specs for Snowflake using Snowpark

import sys
import os
import json
import pandas as pd
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, lit
from snowflake.snowpark.types import StructType, StructField, StringType, VariantType

# Add the src directory to the path
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

from src.onboard_controltable import OnboardControlTable
from src.controltable_spec import ControlTableSpecUtils

print("Snowflake-META Onboarding Example with Snowpark")
print("=" * 50)


In [ ]:
# Step 1: Get Snowpark Session from Notebook Warehouse
# The session is automatically tied to the warehouse assigned to this notebook
# No need for config files or manual connection setup

# Get the current Snowpark session
session = Session.builder.getOrCreate()
print("Snowpark session established successfully!")
print(f"Current warehouse: {session.get_current_warehouse()}")
print(f"Current database: {session.get_current_database()}")
print(f"Current schema: {session.get_current_schema()}")




In [ ]:
# Step 2: Define onboarding parameters
onboarding_params_map = {
    "database": "RAW",
    "schema":"TPCH_SF1",
    "onboarding_file_path":"/onboarding_files/users_onboarding.json",
    "bronze_control_table": "bronze_control_table", 
    "silver_control_table": "silver_control_table",
    "overwrite": "True",
    "env": "dev",
    "version": "v1",
    "import_author": "Maeruf"
}

print("Onboarding parameters configured:")
for key, value in onboarding_params_map.items():
    print(f"  {key}: {value}")

In [ ]:
# Step 3: Create sample onboarding file
# This creates a sample JSON file for demonstration purposes

sample_onboarding_data = [
    {
        "data_flow_id": "user_data_flow_001",
        "data_flow_group": "user_management",
        "source_format": "snowflake",
        "source_details": {
            "source_database_dev": "SOURCE_DB",
            "source_table": "users",
            "source_schema_path": None
        },
        "bronze_database_dev": "SNOWMETA_DB",
        "bronze_table": "bronze_users",
        "bronze_table_path_dev": "/tmp/bronze_users",
        "bronze_table_comment": "Bronze layer for user data",
        "bronze_table_properties": {
            "comment": "Bronze users table",
            "data_retention_time_in_days": 30
        },
        "bronze_partition_columns": "created_date",
        "bronze_cluster_by": ["user_id"],
        "silver_database_dev": "SNOWMETA_DB", 
        "silver_table": "silver_users",
        "silver_table_path_dev": "/tmp/silver_users",
        "silver_table_comment": "Silver layer for user data",
        "silver_table_properties": {
            "comment": "Silver users table",
            "data_retention_time_in_days": 90
        },
        "silver_partition_columns": "created_date",
        "silver_cluster_by": ["user_id"],
        "silver_transformation_json_dev": [
            {
                "target_table": "silver_users",
                "select_exp": ["user_id", "username", "email", "created_date", "updated_date"],
                "where_clause": ["user_id IS NOT NULL"],
                "target_partition_cols": ["created_date"]
            }
        ]
    }
]

# Create the onboarding file
os.makedirs("onboarding_files", exist_ok=True)
with open("onboarding_files/users_onboarding.json", "w") as f:
    json.dump(sample_onboarding_data, f, indent=2)

print("Sample onboarding file created: onboarding_files/users_onboarding.json")
print("Sample data:")
print(json.dumps(sample_onboarding_data, indent=2))


In [ ]:
# Step 4: Initialize and run the onboarding process

# Create the OnboardControlTable instance using Snowpark session
onboard_control_table = OnboardControlTable(
    session=session,
    dict_obj=onboarding_params_map,
    bronze_schema_mapper=None,  # Optional: custom schema mapper
    uc_enabled=False  # Set to True if using Unity Catalog
)

print("OnboardControlTable instance created successfully!")

# Run the onboarding process
try:
    print("Starting onboarding process...")
    onboard_control_table.onboard_controltable_specs()
    print("✅ Onboarding completed successfully!")
except Exception as e:
    print(f"❌ Onboarding failed: {str(e)}")
    raise


In [ ]:
# Step 5: Verify the onboarding results

# Get bronze control table specs
print("Bronze Control Table Specs:")
print("-" * 30)
try:
    bronze_specs = ControlTableSpecUtils.get_bronze_control_table_spec(session)
    for spec in bronze_specs:
        print(f"DataFlow ID: {spec.dataFlowId}")
        print(f"DataFlow Group: {spec.dataFlowGroup}")
        print(f"Source Format: {spec.sourceFormat}")
        print(f"Target Format: {spec.targetFormat}")
        print(f"Version: {spec.version}")
        print(f"Created By: {spec.createdBy}")
        print("-" * 20)
except Exception as e:
    print(f"Error retrieving bronze specs: {str(e)}")

print("\nSilver Control Table Specs:")
print("-" * 30)
try:
    silver_specs = ControlTableSpecUtils.get_silver_control_table_spec(session)
    for spec in silver_specs:
        print(f"DataFlow ID: {spec.dataFlowId}")
        print(f"DataFlow Group: {spec.dataFlowGroup}")
        print(f"Source Format: {spec.sourceFormat}")
        print(f"Target Format: {spec.targetFormat}")
        print(f"Version: {spec.version}")
        print(f"Created By: {spec.createdBy}")
        print("-" * 20)
except Exception as e:
    print(f"Error retrieving silver specs: {str(e)}")


In [ ]:
# Step 6: Clean up

# Snowpark session is automatically managed by the notebook environment
# No need to manually close the session

# Optional: Clean up temporary files
import shutil
try:
    shutil.rmtree("onboarding_files")
    print("Temporary files cleaned up.")
except:
    print("No temporary files to clean up.")

print("\n🎉 Snowflake-META onboarding example completed successfully!")
print("\nNext steps:")
print("1. Review the created control table specs in your Snowflake database")
print("2. Use the specs to create your data pipelines")
print("3. Monitor and maintain your data flows")
print("\nFor more information, see the documentation in the src/ directory.")
